In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/airbnb-listings/exports/csv?lang=en&facet=facet(name%3D%22host_verifications%22%2C%20disjunctive%3Dtrue)&facet=facet(name%3D%22amenities%22%2C%20disjunctive%3Dtrue)&facet=facet(name%3D%22features%22%2C%20disjunctive%3Dtrue)&qv1=(Madrid)&timezone=Europe%2FBerlin&use_labels=true&csv_separator=%3B', sep = ';')

In [ ]:
data.columns

Vemos que Features puedee contener información que puede llegar a modificar el precio del alquiler, decidimos convertir esta información en columnas booleanas para trabajar con ella.

In [ ]:
data['Profile Pic'] = data['Features'].str.contains('Host Has Profile Pic')
data['Identity Verified'] = data['Features'].str.contains('Host Identity Verified')
data['Location Exact'] = data['Features'].str.contains('Is Location Exact')
data['Instant Bookable'] = data['Features'].str.contains('Instant Bookable')
data['License Required'] = data['Features'].str.contains('Requires License')
data['Guest Pic Required'] = data['Features'].str.contains('Require Guest Profile Picture')
data['Superhost'] = data['Features'].str.contains('Host Is Superhost')
data['Guest Phone Verification'] = data['Features'].str.contains('Require Guest Phone Verification')

In [ ]:
data.columns=[col.lower().replace(" ","_") for col in data.columns]

In [ ]:
data.info()

Vemos en forma de cuadro los distintos países que se incluyen en el dataset. Decidimos trabajar con el país que mayor datos tiene.

In [ ]:
sns.countplot(data['country'], palette = "pastel")
plt.title("Countries")

In [ ]:
data = data[data['country']=='Spain']

In [ ]:
data.city.values_counts()

Vemos también que los datos son mayoritariamente de Madrid. Limpiamos para quedarnos solo con datos de Madrid, corrigierndo errores de zipcode y ciudad escrito con otro formato.

In [ ]:
data[data['zipcode'].str.startswith('28'), na=False]

In [ ]:
print(data['zipcode'].unique())

In [ ]:
data=data.replace({'28002\n28002' : '28002', '2802\n28012' : '28012', '28051\n28051': '28051', '280013': '28013'})
print(data['zipcode'].unique())

In [ ]:
data = data.drop(data[data['zipcode']=='28'].index)
data = data.drop(data[data['zipcode']=='2805'].index)
data = data.drop(data[data['zipcode']=='2815'].index)
data = data.drop(data[data['zipcode']=='2804'].index)
print(data['zipcode'].unique())

In [ ]:
data.loc[data['city'] != 'Madrid', 'city'] == 'Madrid'

Con los datos especificos ya limpios y decididos nos deshacemos de columnas con las que no trabajaremos

In [ ]:
data = data.drop(columns = ['listing_url','summary','thumbnail_url','host_total_listings_count', 'medium_url','host_url',
'experiences_offered', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_thumbnail_url',
'host_picture_url','access', 'interaction', 'house_rules', 'host_id', 'host_name', 'host_location', 'host_about', 
'host_listings_count', 'host_verifications', 'neighbourhood', 'city', 'state',  'market', 'smart_location', 'country_code', 
'country', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'weekly_price', 'monthly_price', 'guests_included', 
'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 
'availability_90', 'calendar_last_scraped', 'first_review', 'last_review',  'review_scores_accuracy', 'review_scores_cleanliness',
'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'license', 'jurisdiction_names', 'features', 
'scrape_id', 'last_scraped', 'name', 'space', 'description', 'notes', 'transit', 'street', 'review_scores_rating',
'neighborhood_overview', 'reviews_per_month', 'xl_picture_url'])

In [ ]:
data.info()

Creamos un mapa de correlación para visualizar que columnas tienen una relación mayor y plantear el modelo de regresión

In [ ]:
corr = data.corr(method='kendall')
plt.figure(figsize=(15,8), facecolor = 'white')
sns.heatmap(corr, annot=True,)
data.columns

Hacemos un par de boxplot para comparar la relación que tienen lo alquilados que estan los airbnb si tienen foto de perfil los host o si es superhost.

In [ ]:
sns.boxplot(x='profile_pic', y="availability_365",data=data)

In [ ]:
sns.boxplot(x='superhost', y="availability_365",data=data)

In [ ]:
data.to_csv('data1.csv')
from google.colab import files 
files.download('data1.csv')

Descargamos una version limpia del Dataset para continuar trabajando en Tablau y RStudio.

Guardamos 2 copias, una transformando los NaN a 0 y otra sin contar los NaN, se comparan con boxplot. No se continúa con esta información en Tableau ni en el modelo de regresión

In [ ]:
test_data = data.copy()

In [ ]:
test_data["price"] = data["price"].fillna(0)
test_data["security_deposit"] = data["security_deposit"].fillna(0)
test_data["cleaning_fee"] = data["cleaning_fee"].fillna(0)

In [ ]:
test_data["sum_price"] = test_data.price + test_data.security_deposit + test_data.cleaning_fee

In [ ]:
prueba = data.copy()

In [ ]:
prueba=data.dropna(subset=["security_deposit"])
prueba=prueba.dropna(subset=["cleaning_fee"])
prueba=prueba.dropna(subset=["review_scores_value"])
prueba.info()

In [ ]:
prueba['total_price'] = prueba['price'] + prueba['security_deposit']+ prueba['cleaning_fee']
prueba.info()

In [ ]:
sns.boxplot(x="sum_price", y="room_type", data = test_data)

In [ ]:
sns.boxplot(x="total_price", y="room_type", data = prueba)

In [ ]:
sns.boxplot(x="price", y="room_type", data = test_data)

In [ ]:
sns.boxplot(x="price", y="room_type", data= prueba)